In [ ]:
from random import seed
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp
from lista_doblemente_enlazada import *
import glob
from util import time_algorithm
seed(12345)
np.random.seed(12345)
sns.set_theme()

In [ ]:
#Algoritmo Greedy usado en tp1.py
def verificar_coincidencia_timestamps(t,s):
    solucion = []
    t_ord = ListaDoblementeEnlazada(sorted(t, key = lambda x: x[0] + x[1]))
    
    for timestamp in s:
        flag = False
        
        for nodo in t_ord:
            aproximacion, error = nodo.get_valor()
            inicio = aproximacion - error
            fin = aproximacion + error

            if inicio <= timestamp <= fin:
                flag = True
                t_ord.borrar_elemento(nodo)
                solucion.append((timestamp, aproximacion, error))
                break
        
        if flag == False: # No se seleccionó intervalo -> el problema no tiene solución.
            return None

    return solucion

In [ ]:
# Función get_args para el time_algorithm
def get_args(size):
    archivos = glob.glob(f"Archivos_para_tests_automatizados/{size}-es*.txt")  # Encuentra todos los archivos que coincidan con el patron
    if not archivos:
        raise FileNotFoundError(f"No se encontraron archivos para {size}-es*.txt")

    t_total, s_total = [], []

    for archivo in archivos:
        t, s = procesar_archivo_entrada(archivo)
        t_total.extend(t)
        s_total.extend(s)

    return t_total, s_total

def procesar_archivo_entrada(archivo):
    with open(archivo, 'r') as file:
        lines = file.readlines()

    n = int(lines[1].strip())

    t = [tuple(map(int, x.strip().split(','))) for x in lines[2:2 + n]]
    s = [int(x) for x in lines[2 + n:2 + 2 * n]]
    return t, s

In [ ]:
#Generador de resultados para las mediciones del algoritmo Greedy

x = np.array([
            5, 10, 50, 100, 250, 
            500, 750, 1000, 2500, 
            5000, 7500, 8500, 10000, 
            12000, 13500, 15000, 17500, 
            18000, 20000, 21000, 25000, 
            30000, 32500, 35000, 40000, 
            45000, 50000, 55000, 60000,
            100000, 200000, 300000, 400000, 
            500000, 600000, 700000, 800000, 
            900000, 1000000, 1100000, 1200000, 
            1300000, 1400000, 1500000, 1600000, 
            1700000, 1800000, 1900000, 2000000
            ])
results = time_algorithm(verificar_coincidencia_timestamps, x, get_args)

In [ ]:
#Funciones a usar para las mediciones y aproximaciones dadas por la catedra
f_nlogn = lambda x, c1, c2: c1 * x * np.log(x) + c2 
f_n2 = lambda x, c1, c2: c1 * x**2 + c2

c_nlogn, _ = sp.optimize.curve_fit(f_nlogn, x, [results[n] for n in x])
c_n2, _ = sp.optimize.curve_fit(f_n2, x, [results[n] for n in x])

In [ ]:
#Imprime los resultados de las mediciones en el formato dado
for k, v in results.items():
    print(f"tamaño: {k} tiempo: {v}\n")

In [ ]:
#Imprime la suma de los tiempos promedio
print(f"Suma de los tiempos promedio: ",sum(results.values()))

In [ ]:
#Crea el grafico del algoritmo Greedy por cantidad de elementos en funcion del tiempo (s)
ax: plt.Axes
fig, ax = plt.subplots()
ax.plot(x, [results[i] for i in x], label="Medición")
ax.set_title('Tiempo de ejecución promedio del algoritmo greedy')
ax.set_xlabel('Tamaño del arreglo de timestamps')
ax.set_ylabel('Tiempo de ejecución (s)')
None

In [ ]:
#Crea un grafico como el anterior incluyendo el ajuste a la funcion cuadratica y la lineal logaritmica
ax: plt.Axes
fig, ax = plt.subplots()
ax.plot(x, [results[n] for n in x], "bo-", label="Medición")
ax.plot(x, [f_nlogn(n, c_nlogn[0], c_nlogn[1]) for n in x], 'r--', label="Ajuste $n log(n)$")
ax.plot(x, [f_n2(n, c_n2[0], c_n2[1]) for n in x], 'g--', label="Ajuste $n^2$")
ax.set_yticks(np.linspace(0, max(results.values()) * 1.2, 10))
ax.set_title('Tiempo de ejecución promedio con ajustes')
ax.set_xlabel('Tamaño de datos')
ax.set_ylabel('Tiempo de ejecución (s)')
ax.legend()
None

In [ ]:
#Crea un grafico que muestra el error de ajuste que hay con la cuadratica y la lineal logaritmica
errors_n2 = [np.abs(c_n2[0] * n**2 + c_n2[1] - results[n]) for n in x]
errors_nlogn = [np.abs(c_nlogn[0] * n * np.log(n) + c_nlogn[1] - results[n]) for n in x]
print(f"Error cuadrático total para n^2: {np.sum(np.power(errors_n2, 2))}")

ax: plt.Axes
fig, ax = plt.subplots()
ax.plot(x, errors_n2, label="Ajuste $n^2$")
ax.plot(x, errors_nlogn, label="Ajuste $n \log(n)$")
ax.set_title('Error de ajuste')
ax.set_xlabel('Tamaño de datos')
ax.set_ylabel('Error absoluto (s)')
ax.legend()
None

In [ ]:
#Crea un grafico de la medicion con puntos discretos parecido al visto anteriormente

x_fit = np.linspace(min(x), max(x), 1000)
y_nlogn = f_nlogn(x_fit, *c_nlogn)
y_n2 = f_n2(x_fit, *c_n2)

plt.figure()
plt.scatter(x, [results[n] for n in x], color='r', label='Puntos Discretos', marker='o')
plt.plot(x_fit, y_nlogn, color='g', label='Ajuste O(n log n)')
plt.plot(x_fit, y_n2, color='b', label='Ajuste O(n²)')

plt.title('Tiempo de ejecución promedio con ajustes')
plt.xlabel('Tamaño de datos (n)')
plt.ylabel('Tiempo de ejecucion (s)')
plt.legend()
plt.grid(True)
plt.show()